In [67]:
ACCOUNTING_NUMBERS = {
    '1': '   '+
         '  |'+
         '  |',
    '2': ' _ '+
         ' _|'+
         '|_ ',
    '3': ' _ '+
         ' _|'+
         ' _|',
    '4': ' _ '+
         '|_|'+
         '  |',
    '5': ' _ '+
         '|_ '+
         ' _|',
    '6': ' _ '+
         '|_ '+
         '|_|',
    '7': ' _ '+
         '  |'+
         '  |',
    '8': ' _ '+
         '|_|'+
         '|_|',
    '9': ' _ '+
         '|_|'+
         ' _|',
    '0': ' _ '+
         '| |'+
         '|_|'
}

REVERSED_DB = {v: k for k, v in ACCOUNTING_NUMBERS.items()}


In [68]:
def get_data(filename='printer_output.txt'):
    with open(filename, 'r') as data:
        return data.read()

In [113]:
def get_match(joined_actual_rows):
    unknown = '?'
    value = REVERSED_DB.get(joined_actual_rows, unknown)
    return value

In [253]:
def collect_account_numbers(max_digit = 9):
    entries = get_data().split('\n\n')
    output = []
    collection = []
    for entry in entries:
        collection_inner = []
        start = 0
        end = 3
        number = ''
        joined_actual_rows = ''
        for i in range(max_digit):
            current_digits = []
            actual_rows = []
            for row in entry.split('\n'):
                if len(row) > 0:
                    current_slice = ''.join(row[start:end])
                    actual_rows.append(current_slice)
                    joined_actual_rows = ''.join(actual_rows)
                    current_digits.append(joined_actual_rows)
            
            collection_inner.append(joined_actual_rows)
            value = get_match(joined_actual_rows)
            start += 3
            end += 3
            number += value
        collection.append((collection_inner, number))
        
        start, end = 0, 3
        output.append(number)
        number = ''
    return collection

In [254]:
print(collect_account_numbers())

[(['     |  |', ' _  _||_ ', ' _  _| _|', ' _ |_|  |', ' _ |_  _|', ' _ |_ |_|', ' _   |  |', ' _ |_||_|', ' _ |_| _|'], '123456789'), ([' _ | ||_|', ' _ | ||_|', ' _ | ||_|', ' _ | ||_|', ' _ | ||_|', ' _ | ||_|', ' _ | ||_|', ' _ | ||_|', ' _ | ||_|'], '000000000'), ([' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|'], '666666666'), ([' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |'], '444444444'), ([' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', '   |_| _|'], '99999999?'), (['   |_|  |', ' _ |_| _|', ' _ | ||_|', ' _ | ||_|', ' _ |_ |_|', ' _   |  |', ' _   |  |', '     |  |', ' _ |_  _|'], '?90067715')]


In [441]:
def replace_char_at_index(org_str, index, replacement):
    new_str = org_str
    if index < len(org_str):
        new_str = org_str[0:index] + replacement + org_str[index + 1:]
    return new_str

In [595]:
def fix_validation(illegible_items):
    print(illegible_items)
    replacements = ['|', '_']
    validated = []
    for entry in illegible_items:
        current_number = entry[0][0:9]
        current_string = entry[0]
        new_number = current_number
        if "?" in current_number:
            current_malformed = entry[1][current_number.index("?")]
            new_entry = current_malformed
            for i in range(len(current_malformed)):
                if current_malformed[i] == ' ':
                    for replacement in replacements:
                        backup = new_entry
                        new_entry = replace_char_at_index(new_entry, i, replacement)
                        if get_match(new_entry) != "?":
                            possible_number = get_match(new_entry)
                            print('Found possibility: ' + possible_number)
                            print(new_entry[0:3] + '\n' + new_entry[3:6] + '\n' + new_entry[6:9] + '\n')
                            print(current_number)
                            new_number = replace_char_at_index(new_number, new_number.index("?"), possible_number)
                            print(new_number)
                            print('The checksum is: ' + str(calc_checksum(new_number)))
                        
                                
                                    
                        new_entry = backup
        else:
            pass
        for i in range(len(current_number)):
            pass

In [596]:
def calc_checksum(entry_num):
    valid = True
    sum = 0
    for i in range(1, len(entry_num)+1):
        multi = 10 - i
        num = entry_num[i - 1]
        sum += multi * int(num)
        valid = sum % 11 == 0
    return valid

In [597]:
illegible_items = []
validated = []
def get_validate(entries):
    for entry in entries:
        entry_num = entry[1]
        entry_string = entry[0]
        if "?" in entry_num:
            illegible = True
        else:
            illegible = False
            valid = calc_checksum(entry_num)
                
        if illegible:
            entry_num = f'%s %s' % (entry_num, ' ILL')
            illegible_items.append((entry_num, entry_string))
        elif not valid:
            entry_num = f'%s %s' % (entry_num, ' ERR')
            illegible_items.append((entry_num, entry_string))
        else:
            validated.append(entry_num)
    print(validated)
    fix_validation(illegible_items)
    
    
    

In [598]:
get_validate(entries=collect_account_numbers())


['123456789', '000000000']
[('666666666  ERR', [' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|', ' _ |_ |_|']), ('444444444  ERR', [' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |', ' _ |_|  |']), ('99999999?  ILL', [' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', ' _ |_| _|', '   |_| _|']), ('?90067715  ILL', ['   |_|  |', ' _ |_| _|', ' _ | ||_|', ' _ | ||_|', ' _ |_ |_|', ' _   |  |', ' _   |  |', '     |  |', ' _ |_  _|'])]
Found possibility: 9
 _ 
|_|
 _|

99999999?
999999999
The checksum is: False
Found possibility: 4
 _ 
|_|
  |

?90067715
490067715
The checksum is: False


In [592]:
def save_to_file(entries):
    with open('outfile.txt', 'w') as outfile:
        for entry in entries:
            outfile.write(entry[1] + '\n')
        outfile.close()
    

In [278]:
save_to_file(validated)